In [1]:
import pandas as pd
from pymongo import MongoClient
from collections import defaultdict
from urllib.parse import quote_plus
import numpy as np

| metric                            | source     | descripción                                                                                          |
|-----------------------------------|------------|------------------------------------------------------------------------------------------------------|
| `median_age`                      | INE        | Mediana de edad de la población residente (INE_31304).                                               |
| `youth_share_pop`                 | INE        | Cuota juvenil sobre población total: (18–34 años) / (población total).                                |
| `old_share_pop`                   | INE        | Cuota sénior sobre población total: (65+ años) / (población total).                                   |
| `dependency_ratio`                | INE        | Ratio de dependencia: (65+ años) / (15–64 años).                                                      |
| `generational_balance_index_pop`  | INE        | Índice de equilibrio generacional: `youth_share_pop`  / `old_share_pop`.                              |
| `youth_share_cer`                 | CENSO      | Cuota juvenil sobre censo electoral (CER): (18–34 años) / (total CER).                                |
| `old_share_cer`                   | CENSO      | Cuota sénior sobre censo electoral (CER): (65+ años) / (total CER).                                   |
| `electoral_power_ratio`           | CENSO      | Ratio de poder electoral: (CER ≥65 años) / (CER 18–34 años).                                          |
| `generational_balance_index_cer`  | CENSO      | Índice generacional en el censo: `youth_share_cer` / `old_share_cer`.                                 |
| `projected_median_age`            | EUROSTAT_PEP_NUTS_RAW | Mediana de edad proyectada por año y escenario (EUROSTAT).                                      |
| `projected_youth_share_pop`       | EUROSTAT_PEP_NUTS_RAW | Cuota juvenil proyectada: (18–34) / total proyectado, por escenario.                          |
| `projected_old_share_pop`         | EUROSTAT_PEP_NUTS_RAW | Cuota sénior proyectada: (65+) / total proyectado, por escenario.                             |
| `projected_dependency_ratio`      | EUROSTAT_PEP_NUTS_RAW | Ratio de dependencia proyectado: (65+) / (15–64), por escenario.                               |
| `projected_generational_balance_index_pop` | EUROSTAT_PEP_NUTS_RAW | Índice de equilibrio generacional proyectado: `projected_youth_share_pop`/`projected_old_share_pop`, por escenario. |


In [2]:
# Configurar conexión
usuario = "jalope"
contrasena = "admin"
host = "127.0.0.1"
puerto = "27250"

uri = f"mongodb://{quote_plus(usuario)}:{quote_plus(contrasena)}@{host}:{puerto}/?directConnection=true"
client = MongoClient(uri)
db = client["tfm_db"]

In [12]:
print(db["INE_31304_POBLACION_RAW"].distinct("code_ine_ccaa"))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [13]:
print(db["INE_31304_POBLACION_RAW"].distinct("name_ccaa"))

['Andalucía', 'Aragón', 'Asturias', 'Cantabria', 'Castilla y León', 'Castilla-La Mancha', 'Cataluña', 'Extremadura', 'Galicia', 'Islas Baleares', 'Islas Canarias', 'La Rioja', 'Madrid', 'Murcia', 'Navarra', 'País Vasco', 'Valencia', 'nan']


In [14]:
print(db["INE_31304_POBLACION_RAW"].distinct("code_ine_prov"))

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', 'nan']


In [15]:
print(db["INE_31304_POBLACION_RAW"].distinct("name_prov"))

['A Coruña', 'Albacete', 'Alicante', 'Almería', 'Asturias', 'Badajoz', 'Baleares', 'Barcelona', 'Bizkaia', 'Burgos', 'Cantabria', 'Castellón', 'Ceuta', 'Ciudad Real', 'Cuenca', 'Cáceres', 'Cádiz', 'Córdoba', 'Gipuzkoa', 'Girona', 'Granada', 'Guadalajara', 'Huelva', 'Huesca', 'Jaén', 'La Rioja', 'Las Palmas', 'León', 'Lleida', 'Lugo', 'Madrid', 'Melilla', 'Murcia', 'Málaga', 'Navarra', 'Ourense', 'Palencia', 'Pontevedra', 'Salamanca', 'Santa Cruz de Tenerife', 'Segovia', 'Sevilla', 'Soria', 'Tarragona', 'Teruel', 'Toledo', 'Valencia', 'Valladolid', 'Zamora', 'Zaragoza', 'Álava', 'Ávila']


In [16]:
print(db["INE_31304_POBLACION_RAW"].distinct("year"))

[1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


In [17]:
print(db["INE_31304_POBLACION_RAW"].distinct("sex"))

['F', 'M']


In [18]:
doc = db["INE_31304_POBLACION_RAW"].find_one()
print(doc.keys())

dict_keys(['_id', 'code_ine_prov', 'year', 'age', 'sex', 'population'])


# Mediana de edad por año en España

In [19]:
cursor = db["INE_31304_POBLACION_RAW"].find(
    {}, 
    {"_id":0, "year":1, "age":1, "population":1}
)
df_raw = pd.DataFrame(list(cursor))

# Aggregar para la serie nacional total (sumar M+F y todas las CCAA/Prov)
df_pop = (
    df_raw
    .groupby(["year","age"], as_index=False)["population"]
    .sum()
)

# 3) Etiquetar ámbito nacional y sexo total
df_pop["sex"]           = "T"
df_pop["code_ine_ccaa"] = 0
df_pop["code_ine_prov"] = None

df_pop.head()

,year,age,population,sex,code_ine_ccaa,code_ine_prov
0,1971,0,2.598101e+06,T,0,None
1,1971,1,2.528942e+06,T,0,None
2,1971,2,2.535672e+06,T,0,None
3,1971,3,2.591761e+06,T,0,None
4,1971,4,2.592871e+06,T,0,None


In [20]:
df_pop.sample()

,year,age,population,sex,code_ine_ccaa,code_ine_prov
3471,2005,71,1.617145e+06,T,0,None


In [21]:
df_pop['sex'].unique()

array(['T'], dtype=object)

In [22]:
len(df_pop)

5200

In [24]:
medians = []
for year, grp in df_pop.groupby("year"):
    # población total de ese año
    total = grp["population"].sum()
    # acumulado ordenado por edad
    acum = grp.sort_values("age")["population"].cumsum()
    # edad mínima cuya población acumulada >= 50%
    med_age = grp.loc[acum >= total/2, "age"].iloc[0]
    medians.append({"year": year, "median_age": med_age})

df_median = pd.DataFrame(medians)
print(df_median.head())

   year  median_age
0  1971          29
1  1972          29
2  1973          29
3  1974          29
4  1975          29


In [25]:
df_median

,year,median_age
0,1971,29
1,1972,29
2,1973,29
3,1974,29
4,1975,29
5,1976,29
6,1977,29
7,1978,30
8,1979,30
9,1980,30


In [26]:
# Registros para median_age
records = []
for _, row in df_median.iterrows():
    records.append({
        "source":       "INE",
        "year":         int(row["year"]),
        "region_level": "NAT",
        "region_code":  "ES",
        "metric":       "median_age",
        "scenario":     "OBS",
        "value":        float(row["median_age"])
    })

| Campo          | Valor dado          | Significado                                                              |
| -------------- | ------------------- | ------------------------------------------------------------------------ |
| `source`       | `"INE"`             | La fuente de datos es el Instituto Nacional de Estadística               |
| `year`         | `row["year"]`       | El año al que se refiere la mediana de edad                              |
| `region_level` | `"NAT"`             | Nivel geográfico: nacional (`NAT` = España entera)                       |
| `region_code`  | `"ES"`              | Código del territorio: `'ES'` representa España (homogéneo con EUROSTAT) |
| `metric`       | `"median_age"`      | Indicador que estamos insertando; define qué representa `value`          |
| `scenario`     | `"OBS"`             | Escenario observado (no es proyección)                                   |
| `value`        | `row["median_age"]` | El valor numérico del indicador: la mediana de edad en ese año           |

In [28]:
# Vuelca en DEMOG_INDICATORS_VIEW (limpiando antes si es la primera vez)
view = db["DEMOG_INDICATORS_VIEW"]
view.drop()  
view.insert_many(records)

# 3) Verifica
print("Medianas insertadas:", view.count_documents({"metric":"median_age"}))
print(list(view.find({"metric":"median_age"}).limit(5)))

Medianas insertadas: 52
[{'_id': ObjectId('684b9a29ea38f720dc8e1e7c'), 'source': 'INE', 'year': 1971, 'region_level': 'NAT', 'region_code': 'ES', 'metric': 'median_age', 'scenario': 'OBS', 'value': 29.0}, {'_id': ObjectId('684b9a29ea38f720dc8e1e7d'), 'source': 'INE', 'year': 1972, 'region_level': 'NAT', 'region_code': 'ES', 'metric': 'median_age', 'scenario': 'OBS', 'value': 29.0}, {'_id': ObjectId('684b9a29ea38f720dc8e1e7e'), 'source': 'INE', 'year': 1973, 'region_level': 'NAT', 'region_code': 'ES', 'metric': 'median_age', 'scenario': 'OBS', 'value': 29.0}, {'_id': ObjectId('684b9a29ea38f720dc8e1e7f'), 'source': 'INE', 'year': 1974, 'region_level': 'NAT', 'region_code': 'ES', 'metric': 'median_age', 'scenario': 'OBS', 'value': 29.0}, {'_id': ObjectId('684b9a29ea38f720dc8e1e80'), 'source': 'INE', 'year': 1975, 'region_level': 'NAT', 'region_code': 'ES', 'metric': 'median_age', 'scenario': 'OBS', 'value': 29.0}]


# Personas entre 18 y 34 años / población total

In [29]:
# Población total por año (ya calculada)
total_by_year = df_pop.groupby("year", as_index=False)["population"].sum()
total_by_year = total_by_year.rename(columns={"population": "total_pop"})

In [30]:
# Población joven por año (18 ≤ edad ≤ 34)
mask = df_pop["age"].between(18, 34)
df_young = df_pop[mask]
young_by_year = df_young.groupby("year", as_index=False)["population"].sum()
young_by_year = young_by_year.rename(columns={"population": "youth_pop"})

In [31]:
# Unir y calcular proporción
df_quotas = pd.merge(total_by_year, young_by_year, on="year")
df_quotas["youth_share_pop"] = df_quotas["youth_pop"] / df_quotas["total_pop"]

In [33]:
df_quotas

,year,total_pop,youth_pop,youth_share_pop
0,1971,1.357638e+08,3.175247e+07,0.233880
1,1972,1.372513e+08,3.198025e+07,0.233005
2,1973,1.387908e+08,3.241028e+07,0.233519
3,1974,1.402821e+08,3.309060e+07,0.235886
4,1975,1.418234e+08,3.378881e+07,0.238246
5,1976,1.432982e+08,3.420364e+07,0.238689
6,1977,1.448076e+08,3.493461e+07,0.241248
7,1978,1.462165e+08,3.552354e+07,0.242952
8,1979,1.475124e+08,3.597640e+07,0.243887
9,1980,1.486774e+08,3.640041e+07,0.244828


In [34]:
records = []
for _, row in df_quotas.iterrows():
    records.append({
        "source":       "INE",
        "year":         int(row["year"]),
        "region_level": "NAT",
        "region_code":  "ES",
        "metric":       "youth_share_pop",
        "scenario":     "OBS",
        "value":        float(row["youth_share_pop"])
    })

| Campo          | Valor                                             |
| -------------- | ------------------------------------------------- |
| `source`       | `"INE"`                                           |
| `year`         | año de la fila en `df_quotas`                     |
| `region_level` | `"NAT"`                                           |
| `region_code`  | `"ES"`                                            |
| `metric`       | `"youth_share_pop"`                               |
| `scenario`     | `"OBS"`                                           |
| `value`        | el valor de `youth_share_pop` (float entre 0 y 1) |


In [35]:
view.insert_many(records)
print("Cuota juvenil insertada:", view.count_documents({"metric":"youth_share_pop"}))

Cuota juvenil insertada: 52


# Tasa de dependencia

In [36]:
# Población “old” (65+)
old_mask = df_pop["age"] >= 65
old_by_year = (
    df_pop[old_mask]
    .groupby("year", as_index=False)["population"]
    .sum()
    .rename(columns={"population": "old_pop"})
)

In [37]:
# Población trabajadora (15–64)
work_mask = df_pop["age"].between(15, 64)
work_by_year = (
    df_pop[work_mask]
    .groupby("year", as_index=False)["population"]
    .sum()
    .rename(columns={"population": "work_pop"})
)

In [38]:
# Unir y calcular ratio
df_dep = pd.merge(old_by_year, work_by_year, on="year")
df_dep["dependency_ratio"] = df_dep["old_pop"] / df_dep["work_pop"]

# Verifica
df_dep

,year,old_pop,work_pop,dependency_ratio
0,1971,1.248006e+07,8.535660e+07,0.146211
1,1972,1.278154e+07,8.617758e+07,0.148316
2,1973,1.313192e+07,8.709628e+07,0.150775
3,1974,1.345844e+07,8.802455e+07,0.152894
4,1975,1.379189e+07,8.895198e+07,0.155049
5,1976,1.416555e+07,8.987308e+07,0.157617
6,1977,1.452171e+07,9.081732e+07,0.159900
7,1978,1.489043e+07,9.180284e+07,0.162200
8,1979,1.524648e+07,9.288711e+07,0.164140
9,1980,1.561144e+07,9.404095e+07,0.166007


La **tasa de dependencia de la vejez**(`dependency_ratio`) se interpreta así:

* Es el cociente

  $$
     \frac{\text{población de 65+ años}}{\text{población de 15–64 años}}
  $$

  y suele expresarse como un número (no porcentaje).

* **Ejemplo**: si para 2022 obtienes

  ```
  old_pop   = 10 000 000
  work_pop  = 25 000 000
  dependency_ratio = 10 000 000 / 25 000 000 = 0.40
  ```

  Significa que, por cada persona en edad de trabajar (15–64), hay 0.40 personas de 65 o más.
  O dicho de otro modo, **40 personas mayores por cada 100 en edad de trabajar**.

In [39]:
# Preparamos los registros
records = []
for _, row in df_dep.iterrows():
    records.append({
        "source":       "INE",                  # origen
        "year":         int(row["year"]),       # año
        "region_level": "NAT",                  # nacional
        "region_code":  "ES",                   # código España
        "metric":       "dependency_ratio",     # tasa dependencia
        "scenario":     "OBS",                  # observado
        "value":        float(row["dependency_ratio"])  # valor calculado
    })

| Campo          | Valor asignado            | Descripción                                                                 |
| -------------- | ------------------------- | --------------------------------------------------------------------------- |
| `source`       | `"INE"`                   | El dato proviene del Instituto Nacional de Estadística.                     |
| `year`         | `row["year"]`             | Año de referencia del indicador (1971–2022).                                |
| `region_level` | `"NAT"`                   | Nivel geográfico: nacional (no CCAA ni provincias).                         |
| `region_code`  | `"ES"`                    | Código del territorio: `'ES'` representa a España.                          |
| `metric`       | `"dependency_ratio"`      | Nombre del indicador: proporción de mayores sobre población activa.         |
| `scenario`     | `"OBS"`                   | Escenario observado, no proyección.                                         |
| `value`        | `row["dependency_ratio"]` | Valor calculado: población 65+ / población 15–64 (float entre 0.15 y 0.50). |


In [40]:
# Insertamos
view.insert_many(records)
print("Dependencia insertada:", view.count_documents({"metric":"dependency_ratio"}))

Dependencia insertada: 52


# Poder electoral relativo sénior vs joven

**Fórmula**:

$$
\text{electoral\_power\_ratio} = \frac{\text{personas de 65+ en el censo}}{\text{personas de 18 a 34 en el censo}}
$$

Este indicador mide cuántos votantes mayores hay por cada votante joven.

In [42]:
coll = db["INE_59777_33525_CENSO_RAW"]

df_censo = pd.DataFrame(list(coll.find({}, {"_id":0})))

In [44]:
df_censo['age_range'].unique()

array(['Total', '18 Y 19', '20 A 24', '25 A 29', '30 A 34', '35 A 39',
       '40 A 44', '45 A 49', '50 A 54', '55 A 59', '60 A 64', '65 A 69',
       '70 A 74', '75 A 79', '80 A 84', '85 Y Más'], dtype=object)

In [49]:
df_censo

,year,age_range,sex,census
0,2019,Total,F,17996382.0
1,2019,Total,M,16874100.0
2,2019,Total,T,34870482.0
3,2019,18 Y 19,F,406085.0
4,2019,18 Y 19,M,429591.0
...,...,...,...,...
91,2023,80 A 84,M,531107.0
92,2023,80 A 84,T,1280260.0
93,2023,85 Y Más,F,1098881.0
94,2023,85 Y Más,M,564398.0


In [45]:
# Rango de edad definido por etiquetas
young_ranges = ['18 Y 19', '20 A 24', '25 A 29', '30 A 34']
senior_ranges = ['65 A 69', '70 A 74', '75 A 79', '80 A 84', '85 Y Más']

In [51]:
df_censo_tot = df_censo[df_censo["sex"] == "T"]

In [52]:
# Censo juvenil (18–34) usando df_censo_tot
df_young = (
    df_censo_tot[df_censo_tot["age_range"].isin(young_ranges)]
    .groupby("year", as_index=False)["census"]
    .sum()
    .rename(columns={"census": "young"})
)

In [53]:
# Censo sénior (65+) usando df_censo_tot
df_senior = (
    df_censo_tot[df_censo_tot["age_range"].isin(senior_ranges)]
    .groupby("year", as_index=False)["census"]
    .sum()
    .rename(columns={"census": "senior"})
)

In [54]:
# Merge y ratio
df_epower = pd.merge(df_young, df_senior, on="year")
df_epower["electoral_power_ratio"] = df_epower["senior"] / df_epower["young"]

In [55]:
df_epower

,year,young,senior,electoral_power_ratio
0,2019,7111463.0,8982950.0,1.263165
1,2023,7138375.0,9480388.0,1.328088


In [56]:
view

Collection(Database(MongoClient(host=['127.0.0.1:27250'], document_class=dict, tz_aware=False, connect=True, directconnection=True), 'tfm_db'), 'DEMOG_INDICATORS_VIEW')

| Campo           | Valor                        | Descripción                                                                 |
|-----------------|------------------------------|-----------------------------------------------------------------------------|
| `source`        | `"CENSO"`                    | El dato proviene del censo electoral del INE                               |
| `year`          | `row["year"]`                | Año del censo (2019, 2023, etc.)                                           |
| `region_level`  | `"NAT"`                      | Nivel territorial: nacional                                                |
| `region_code`   | `None`                       | Código del territorio: se deja vacío para el total nacional                |
| `metric`        | `"electoral_power_ratio"`    | Nombre del indicador: razón entre censados sénior (65+) y jóvenes (18–34) |
| `scenario`      | `"OBS"`                      | Escenario observado, no proyección                                         |
| `value`         | `row["electoral_power_ratio"]` | Valor numérico del indicador para ese año (float)                       |


In [57]:
# Preparar registros
records = []
for _, row in df_epower.iterrows():
    records.append({
        "source":       "CENSO",                    # datos de censo electoral
        "year":         int(row["year"]),           # año de la elección
        "region_level": "NAT",                      # ámbito nacional
        "region_code":  None,                       # código 00 → nacional
        "metric":       "electoral_power_ratio",    # ratio 65+/18–34 en censo
        "scenario":     "OBS",                      # observado
        "value":        float(row["electoral_power_ratio"])
    })

# Insertar en la vista
view.insert_many(records)
print("Electoral power ratio insertado:",
      view.count_documents({"metric":"electoral_power_ratio"}))

Electoral power ratio insertado: 2


# Cuota senior

**Definición**

$$
\text{old\_share\_pop} = \frac{\text{población de ≥65 años}}{\text{población total}}
$$

In [59]:
# Población sénior (65+)
old_by_year = (
    df_pop[df_pop["age"] >= 65]
    .groupby("year", as_index=False)["population"]
    .sum()
    .rename(columns={"population": "old_pop"})
)

In [60]:
# Población total (ya lo teníamos en total_by_year)
total_by_year = df_pop.groupby("year", as_index=False)["population"] \
                      .sum() \
                      .rename(columns={"population": "total_pop"})

# Unir y calcular cuota sénior
df_old = pd.merge(total_by_year, old_by_year, on="year")
df_old["old_share_pop"] = df_old["old_pop"] / df_old["total_pop"]

In [61]:
df_old

,year,total_pop,old_pop,old_share_pop
0,1971,1.357638e+08,1.248006e+07,0.091925
1,1972,1.372513e+08,1.278154e+07,0.093125
2,1973,1.387908e+08,1.313192e+07,0.094617
3,1974,1.402821e+08,1.345844e+07,0.095938
4,1975,1.418234e+08,1.379189e+07,0.097247
5,1976,1.432982e+08,1.416555e+07,0.098854
6,1977,1.448076e+08,1.452171e+07,0.100283
7,1978,1.462165e+08,1.489043e+07,0.101838
8,1979,1.475124e+08,1.524648e+07,0.103357
9,1980,1.486774e+08,1.561144e+07,0.105002


| Campo          | Valor                  | Descripción                                               |
| -------------- | ---------------------- | --------------------------------------------------------- |
| `source`       | `"INE"`                | Origen de los datos (Instituto Nacional de Estadística).  |
| `year`         | `row["year"]`          | Año de la observación (1971–2022).                        |
| `region_level` | `"NAT"`                | Nivel territorial: nacional (España entera).              |
| `region_code`  | `"ES"`                 | Código territorial: “ES” para España.                     |
| `metric`       | `"old_share_pop"`      | Indicador: proporción de ≥65 años sobre población total.  |
| `scenario`     | `"OBS"`                | Escenario observado (no es proyección).                   |
| `value`        | `row["old_share_pop"]` | Valor del indicador (float entre 0 y 1, p.ej. 0.20–0.30). |


In [62]:
# Preparar registros
records = []
for _, row in df_old.iterrows():
    records.append({
        "source":       "INE",               # Instituto Nacional de Estadística
        "year":         int(row["year"]),    # Año de referencia
        "region_level": "NAT",               # Nivel territorial: nacional
        "region_code":  "ES",                # Código España
        "metric":       "old_share_pop",     # Cuota sénior sobre población total
        "scenario":     "OBS",               # Escenario observado
        "value":        float(row["old_share_pop"])  # Valor de la cuota sénior
    })

# Insertar en la vista
view.insert_many(records)
print("old_share_pop insertado:", view.count_documents({"metric":"old_share_pop"}))

old_share_pop insertado: 52


# Índice de equilibrio generacional sobre población

**Definición**

$$
\text{generational\_balance\_index\_pop} = \frac{\text{youth\_share\_pop}}{\text{old\_share\_pop}}
$$

In [64]:
# Unir youth_share_pop y old_share_pop por año
df_balance = pd.merge(
    df_quotas[['year','youth_share_pop']],
    df_old[['year','old_share_pop']],
    on='year'
)

In [65]:
# Calcular el índice
df_balance['generational_balance_index_pop'] = (
    df_balance['youth_share_pop'] / df_balance['old_share_pop']
)

In [66]:
df_balance

,year,youth_share_pop,old_share_pop,generational_balance_index_pop
0,1971,0.233880,0.091925,2.544256
1,1972,0.233005,0.093125,2.502066
2,1973,0.233519,0.094617,2.468054
3,1974,0.235886,0.095938,2.458724
4,1975,0.238246,0.097247,2.449904
5,1976,0.238689,0.098854,2.414565
6,1977,0.241248,0.100283,2.405682
7,1978,0.242952,0.101838,2.385662
8,1979,0.243887,0.103357,2.359653
9,1980,0.244828,0.105002,2.331650


# * **> 1** (p.ej. 2.5) en los primeros años: la **cuota juvenil** (18–34) era 2,5 veces la **cuota sénior** (65+), es decir, había 2,5 jóvenes por cada sénior en la población total.
* **= 1** en el punto de cruce: juventud y sénior representaban igual proporción de la población.
* **< 1** (p.ej. 0.9) en los años más recientes: la **cuota juvenil** es sólo el 90 % de la **cuota sénior**, lo que refleja que los mayores ya superan en proporción a los jóvenes.

| Campo          | Valor                                   | Descripción                                                                           |
| -------------- | --------------------------------------- | ------------------------------------------------------------------------------------- |
| `source`       | `"INE"`                                 | Datos del Instituto Nacional de Estadística                                           |
| `year`         | `row["year"]`                           | Año de referencia                                                                     |
| `region_level` | `"NAT"`                                 | Nivel geográfico nacional                                                             |
| `region_code`  | `"ES"`                                  | Código del territorio: España                                                         |
| `metric`       | `"generational_balance_index_pop"`      | Nombre del indicador: ratio de cuota juvenil sobre cuota sénior en la población total |
| `scenario`     | `"OBS"`                                 | Escenario observado                                                                   |
| `value`        | `row["generational_balance_index_pop"]` | Valor numérico del índice (float)                                                     |


In [67]:
records = []
for _, row in df_balance.iterrows():
    records.append({
        "source":       "INE",                            # Instituto Nacional de Estadística
        "year":         int(row["year"]),                 # Año de cálculo
        "region_level": "NAT",                            # Nivel nacional
        "region_code":  "ES",                             # Código España
        "metric":       "generational_balance_index_pop", # Índice equilibrio generacional (pob.)
        "scenario":     "OBS",                            # Observado
        "value":        float(row["generational_balance_index_pop"])
    })

# Insertar
view.insert_many(records)
print("Inserted generational_balance_index_pop:",
      view.count_documents({"metric":"generational_balance_index_pop"}))

Inserted generational_balance_index_pop: 52


# `youth_share_cer` (cuota juvenil sobre censo CER)

In [68]:
young_ranges

['18 Y 19', '20 A 24', '25 A 29', '30 A 34']

In [69]:
# Filtrar solo sexo total CER
df_cer_tot = df_censo[df_censo["sex"] == "T"]

# Población total CER por año
total_cer = (
    df_cer_tot
    .groupby("year", as_index=False)["census"]
    .sum()
    .rename(columns={"census": "total_cer"})
)

In [70]:
# Población joven CER (18–34) por año
youth_cer = (
    df_cer_tot[df_cer_tot["age_range"].isin(young_ranges)]
    .groupby("year", as_index=False)["census"]
    .sum()
    .rename(columns={"census": "youth_cer"})
)

# Unir y calcular cuota juvenil
df_cer_quota = pd.merge(total_cer, youth_cer, on="year")
df_cer_quota["youth_share_cer"] = df_cer_quota["youth_cer"] / df_cer_quota["total_cer"]

df_cer_quota

,year,total_cer,youth_cer,youth_share_cer
0,2019,69740964.0,7111463.0,0.101970
1,2023,70281762.0,7138375.0,0.101568


| Campo          | Valor                    | Descripción                                                           |
| -------------- | ------------------------ | --------------------------------------------------------------------- |
| `source`       | `"CENSO"`                | Fuente: censo electoral del INE                                       |
| `year`         | `row["year"]`            | Año del censo (2019, 2023)                                            |
| `region_level` | `"NAT"`                  | Nivel territorial: nacional                                           |
| `region_code`  | `None`                   | Código: vacío para el total nacional                                  |
| `metric`       | `"youth_share_cer"`      | Indicador: proporción de electores 18–34 sobre total de electores CER |
| `scenario`     | `"OBS"`                  | Escenario observado                                                   |
| `value`        | `row["youth_share_cer"]` | Valor numérico del indicador (float entre 0 y 1)                      |


In [71]:
# Preparar registros para youth_share_cer
records = []
for _, row in df_cer_quota.iterrows():
    records.append({
        "source":       "CENSO",            # Censo Electoral
        "year":         int(row["year"]),   # Año de las elecciones
        "region_level": "NAT",              # Nivel nacional
        "region_code":  None,               # Total nacional
        "metric":       "youth_share_cer",  # Cuota juvenil sobre censo
        "scenario":     "OBS",              # Observado
        "value":        float(row["youth_share_cer"])
    })

# Insertar en la vista
view.insert_many(records)
print("youth_share_cer insertado:",
      view.count_documents({"metric":"youth_share_cer"}))

youth_share_cer insertado: 2


# `old_share_cer` – Cuota sénior sobre censo CER

**Definición**

$$
\text{old\_share\_cer} = \frac{\text{CER ≥65 años}}{\text{total CER}}
$$

In [72]:
senior_ranges

['65 A 69', '70 A 74', '75 A 79', '80 A 84', '85 Y Más']

In [73]:
# Filtrar sólo sexo total
df_cer_tot = df_censo[df_censo["sex"] == "T"]

# 1) Total CER ya en total_cer
# 2) Sénior CER por año
senior_cer = (
    df_cer_tot[df_cer_tot["age_range"].isin(senior_ranges)]
    .groupby("year", as_index=False)["census"]
    .sum()
    .rename(columns={"census": "senior_cer"})
)

In [74]:
# Unir con total_cer y calcular cuota
df_cer_old = pd.merge(total_cer, senior_cer, on="year")
df_cer_old["old_share_cer"] = df_cer_old["senior_cer"] / df_cer_old["total_cer"]

In [75]:
df_cer_old

,year,total_cer,senior_cer,old_share_cer
0,2019,69740964.0,8982950.0,0.128805
1,2023,70281762.0,9480388.0,0.134891


| Campo          | Valor                  | Descripción                                                              |
| -------------- | ---------------------- | ------------------------------------------------------------------------ |
| `source`       | `"CENSO"`              | Fuente de datos: censo electoral del INE                                 |
| `year`         | `row["year"]`          | Año del censo (2019, 2023, etc.)                                         |
| `region_level` | `"NAT"`                | Nivel territorial: nacional                                              |
| `region_code`  | `None`                 | Código del territorio: vacío para agregado nacional                      |
| `metric`       | `"old_share_cer"`      | Indicador: proporción de electores ≥65 años sobre total de electores CER |
| `scenario`     | `"OBS"`                | Escenario observado (no proyección)                                      |
| `value`        | `row["old_share_cer"]` | Valor numérico del indicador (float entre 0 y 1)                         |


In [76]:
# Preparar registros para old_share_cer
records = []
for _, row in df_cer_old.iterrows():
    records.append({
        "source":       "CENSO",           # Origen: censo electoral
        "year":         int(row["year"]),  # Año de la observación
        "region_level": "NAT",             # Nivel nacional
        "region_code":  None,              # Total nacional
        "metric":       "old_share_cer",   # Cuota sénior sobre censo CER
        "scenario":     "OBS",             # Escenario observado
        "value":        float(row["old_share_cer"])  # Valor del indicador
    })

# Insertar en la vista
view.insert_many(records)
print("old_share_cer insertado:", view.count_documents({"metric":"old_share_cer"}))

old_share_cer insertado: 2


# generational_balance_index_cer – Índice generacional en el censo electoral

**Definición**

$$
\text{generational\_balance\_index\_cer} = \frac{\text{youth\_share\_cer}}{\text{old\_share\_cer}}
$$

In [79]:
# Unir youth_share_cer y old_share_cer por año
df_balance_cer = pd.merge(
    df_cer_quota[['year', 'youth_share_cer']],
    df_cer_old[['year', 'old_share_cer']],
    on='year'
)

In [80]:
# Calcular el índice
df_balance_cer['generational_balance_index_cer'] = (
    df_balance_cer['youth_share_cer'] / df_balance_cer['old_share_cer']
)

In [81]:
df_balance_cer

,year,youth_share_cer,old_share_cer,generational_balance_index_cer
0,2019,0.101970,0.128805,0.791662
1,2023,0.101568,0.134891,0.752962


| Campo          | Valor                                   | Descripción                                                             |
| -------------- | --------------------------------------- | ----------------------------------------------------------------------- |
| `source`       | `"CENSO"`                               | Fuente de datos: censo electoral del INE                                |
| `year`         | `row["year"]`                           | Año del censo (2019, 2023, etc.)                                        |
| `region_level` | `"NAT"`                                 | Nivel territorial: nacional                                             |
| `region_code`  | `None`                                  | Código: vacío para total nacional                                       |
| `metric`       | `"generational_balance_index_cer"`      | Indicador: ratio de cuota juvenil vs cuota sénior en el censo electoral |
| `scenario`     | `"OBS"`                                 | Escenario observado (no es proyección)                                  |
| `value`        | `row["generational_balance_index_cer"]` | Valor numérico del indicador (float)                                    |


In [82]:
# Preparar registros para generational_balance_index_cer
records = []
for _, row in df_balance_cer.iterrows():
    records.append({
        "source":       "CENSO",                             # Censo electoral
        "year":         int(row["year"]),                    # Año de la observación
        "region_level": "NAT",                               # Nivel nacional
        "region_code":  None,                                # Total nacional
        "metric":       "generational_balance_index_cer",    # Índice generacional en censo
        "scenario":     "OBS",                               # Escenario observado
        "value":        float(row["generational_balance_index_cer"])
    })

# Insertar en la vista
view.insert_many(records)
print("generational_balance_index_cer insertado:",
      view.count_documents({"metric":"generational_balance_index_cer"}))

generational_balance_index_cer insertado: 2


# projected_median_age – Mediana de edad proyectada por escenario
**Definición**
Para cada combinación (year, scenario) en tu DataFrame de proyecciones (df_proj_norm), calculamos la mediana de edad de manera análoga a como lo hicimos con los datos observados, pero ahora filtrando solo:
- cod_nuts == "Spain" (nivel nacional)
- sex == "T"
- Agrupando por year y scenario.

In [4]:
# Traemos sólo España (cod_nuts='Spain') y sexo total (T)
cursor = db["EUROSTAT_PEP_NUTS_RAW"].find(
    {"cod_nuts":"Spain", "sex":"T"},
    {"_id":0,"year":1,"age":1,"scenario":1,"population":1}
)
df_proj_norm = pd.DataFrame(list(cursor))
print("Proyecciones cargadas:", df_proj_norm.shape)
print(df_proj_norm.head())

Proyecciones cargadas: (51192, 4)
   year     age scenario  population
0  2022  1 year      BSL      345218
1  2023  1 year      BSL      348316
2  2024  1 year      BSL      343863
3  2025  1 year      BSL      344192
4  2026  1 year      BSL      344898


In [8]:
df_proj_norm['age'].unique()

array(['1 year', '10 years', '11 years', '12 years', '13 years',
       '14 years', '15 years', 'From 15 to 64 years',
       'From 15 to 74 years', '16 years', '17 years', '18 years',
       '19 years', '2 years', '20 years', 'From 20 to 64 years',
       '21 years', '22 years', '23 years', '24 years', '25 years',
       '26 years', '27 years', '28 years', '29 years', '3 years',
       '30 years', '31 years', '32 years', '33 years', '34 years',
       '35 years', '36 years', '37 years', '38 years', '39 years',
       '4 years', '40 years', '41 years', '42 years', '43 years',
       '44 years', '45 years', '46 years', '47 years', '48 years',
       '49 years', '5 years', '50 years', '51 years', '52 years',
       '53 years', '54 years', '55 years', '56 years', '57 years',
       '58 years', '59 years', '6 years', '60 years', '61 years',
       '62 years', '63 years', '64 years', '65 years', '66 years',
       '67 years', '68 years', '69 years', '7 years', '70 years',
       '71 years',

| Etiqueta                               | Acción        | Mapeo                               |
| -------------------------------------- | ------------- | ----------------------------------- |
| **“Less than 1 year”**                 | **Conservar** | Mapear a `0`                        |
| **“1 year”, “2 years”, …, “99 years”** | **Conservar** | Extraer el número → edad entera     |
| **“65 years or over”**                 | **Descartar** | (open-ended, no sirve para mediana) |
| **“75 years or over”**                 | **Descartar** |                                     |
| **“80 years or over”**                 | **Descartar** |                                     |
| **“From 15 to 64 years”**              | **Descartar** | (agrupación, no individual)         |
| **“From 15 to 74 years”**              | **Descartar** |                                     |
| **“Less than 15 years”**               | **Descartar** |                                     |
| **“Less than 20 years”**               | **Descartar** |                                     |


In [11]:
# Filtrar solo filas con edad numérica o "Less than 1 year"
mask = (
    df_proj_norm["age"].str.match(r"^\d+ years?$") |
    (df_proj_norm["age"] == "Less than 1 year")
)
df_proj_norm = df_proj_norm[mask].copy()

# Mapear a número
df_proj_norm["age"] = (
    df_proj_norm["age"]
    .replace({"Less than 1 year": "0"})
    .str.extract(r"(\d+)")
    .astype(int)
)

In [12]:
print(df_proj_norm["age"].unique())
df_proj_norm.dtypes

[ 1 10 11 12 13 14 15 16 17 18 19  2 20 21 22 23 24 25 26 27 28 29  3 30
 31 32 33 34 35 36 37 38 39  4 40 41 42 43 44 45 46 47 48 49  5 50 51 52
 53 54 55 56 57 58 59  6 60 61 62 63 64 65 66 67 68 69  7 70 71 72 73 74
 75 76 77 78 79  8 80 81 82 83 84 85 86 87 88 89  9 90 91 92 93 94 95 96
 97 98 99  0]


year           int64
age            int64
scenario      object
population     int64
dtype: object

In [11]:
# mediana proyectada
meds = []
for (yr, sc), grp in df_proj_norm.groupby(['year','scenario'], sort=False):
    total_pop = grp['population'].sum()
    acum      = grp.sort_values('age')['population'].cumsum()
    med_age   = grp.loc[acum >= total_pop/2, 'age'].iloc[0]
    meds.append({
        'year':       int(yr),
        'scenario':   sc,
        'median_age': med_age
    })

df_proj_median = pd.DataFrame(meds)
print(df_proj_median.head())

   year scenario  median_age
0  2022      BSL          45
1  2023      BSL          45
2  2024      BSL          45
3  2025      BSL          46
4  2026      BSL          46


In [12]:
view   = db["DEMOG_INDICATORS_VIEW"]

# Preparar registros para projected_median_age
records = []
for _, row in df_proj_median.iterrows():
    records.append({
        "source":       "EUROSTAT_PEP_NUTS_RAW",  # Proyecciones Eurostat
        "year":         int(row["year"]),         # Año de la proyección
        "region_level": "NAT",                    # Nivel nacional
        "region_code":  "ES",                     # España
        "metric":       "projected_median_age",   # Mediana proyectada
        "scenario":     row["scenario"],          # Escenario (BSL, LFRT, etc.)
        "value":        float(row["median_age"])  # Edad mediana proyectada
    })

# Insertar en la vista
view.insert_many(records)
print("projected_median_age insertado:",
      view.count_documents({"metric":"projected_median_age"}))

projected_median_age insertado: 474


# `projected_youth_share_pop` – Cuota juvenil proyectada

In [3]:
view   = db["DEMOG_INDICATORS_VIEW"]

**Definición**

$$
\text{projected\_youth\_share\_pop} = \frac{\text{proyección de población 18–34}}{\text{proyección de población total}}
$$

In [5]:
# Población total proyectada por año y escenario
total_proj = (
    df_proj_norm
    .groupby(["year","scenario"], as_index=False)["population"]
    .sum()
    .rename(columns={"population":"total_pop"})
)

In [13]:
# Población juvenil proyectada (18–34) por año y escenario
young_mask = df_proj_norm["age"].between(18, 34)
young_proj = (
    df_proj_norm[young_mask]
    .groupby(["year","scenario"], as_index=False)["population"]
    .sum()
    .rename(columns={"population":"youth_pop"})
)

In [14]:
# Unir y calcular cuota juvenil proyectada
df_proj_quota = pd.merge(total_proj, young_proj, on=["year","scenario"])
df_proj_quota["projected_youth_share_pop"] = (
    df_proj_quota["youth_pop"] / df_proj_quota["total_pop"]
)

In [15]:
df_proj_quota.head()

,year,scenario,total_pop,youth_pop,projected_youth_share_pop
0,2022,BSL,176446005,8635116,0.048939
1,2022,HMIGR,176446005,8635116,0.048939
2,2022,LFRT,176446005,8635116,0.048939
3,2022,LMIGR,176446005,8635116,0.048939
4,2022,LMRT,176446005,8635116,0.048939


| Campo          | Valor                              | Descripción                                                    |
| -------------- | ---------------------------------- | -------------------------------------------------------------- |
| `source`       | `"EUROSTAT_PEP_NUTS_RAW"`          | Datos de proyecciones Eurostat (EUROPOP2023)                   |
| `year`         | `row["year"]`                      | Año de la proyección (2022–2100)                               |
| `region_level` | `"NAT"`                            | Nivel territorial: nacional                                    |
| `region_code`  | `"ES"`                             | Código del territorio: España                                  |
| `metric`       | `"projected_youth_share_pop"`      | Indicador: cuota juvenil proyectada (18–34 / total)            |
| `scenario`     | `row["scenario"]`                  | Escenario de proyección (BSL, LFRT, LMRT, HMIGR, LMIGR, NMIGR) |
| `value`        | `row["projected_youth_share_pop"]` | Valor numérico del indicador (float entre 0 y 1)               |


In [16]:
# Preparar registros para projected_youth_share_pop
records = []
for _, row in df_proj_quota.iterrows():
    records.append({
        "source":       "EUROSTAT_PEP_NUTS_RAW",   # Proyecciones Eurostat
        "year":         int(row["year"]),          # Año de la proyección
        "region_level": "NAT",                     # Nivel territorial: nacional
        "region_code":  "ES",                      # España
        "metric":       "projected_youth_share_pop",  # Cuota juvenil proyectada
        "scenario":     row["scenario"],           # Escenario (BSL, LFRT…)
        "value":        float(row["projected_youth_share_pop"])  # Proporción 18–34 / total
    })

# Insertar en la vista
view.insert_many(records)
print("projected_youth_share_pop insertado:",
      view.count_documents({"metric":"projected_youth_share_pop"}))

projected_youth_share_pop insertado: 474


# `projected_old_share_pop` – Cuota sénior proyectada

**Definición**

$$
\text{projected\_old\_share\_pop} = \frac{\text{proyección de población ≥65 años}}{\text{proyección de población total}}
$$

In [18]:
# 1. Población total proyectada ya la tenemos en total_proj
# 2. Población sénior proyectada (65+) por año y escenario
senior_proj = (
    df_proj_norm[df_proj_norm["age"] >= 65]
    .groupby(["year","scenario"], as_index=False)["population"]
    .sum()
    .rename(columns={"population":"old_pop"})
)

# Unir con total_proj y calcular cuota sénior proyectada
df_proj_old = pd.merge(total_proj, senior_proj, on=["year","scenario"])
df_proj_old["projected_old_share_pop"] = df_proj_old["old_pop"] / df_proj_old["total_pop"]

In [19]:
df_proj_old

,year,scenario,total_pop,old_pop,projected_old_share_pop
0,2022,BSL,176446005,9512343,0.053911
1,2022,HMIGR,176446005,9512343,0.053911
2,2022,LFRT,176446005,9512343,0.053911
3,2022,LMIGR,176446005,9512343,0.053911
4,2022,LMRT,176446005,9512343,0.053911
...,...,...,...,...,...
469,2100,HMIGR,197182191,17631391,0.089417
470,2100,LFRT,146248513,14929455,0.102083
471,2100,LMIGR,137328786,13033808,0.094910
472,2100,LMRT,170428397,16139056,0.094697


| Campo          | Valor                            | Descripción                                                    |
| -------------- | -------------------------------- | -------------------------------------------------------------- |
| `source`       | `"EUROSTAT_PEP_NUTS_RAW"`        | Colección de proyecciones Eurostat (EUROPOP2023)               |
| `year`         | `row["year"]`                    | Año de la proyección (2022–2100)                               |
| `region_level` | `"NAT"`                          | Nivel territorial: nacional                                    |
| `region_code`  | `"ES"`                           | Código del territorio: España                                  |
| `metric`       | `"projected_old_share_pop"`      | Indicador: cuota sénior proyectada (≥65 / total)               |
| `scenario`     | `row["scenario"]`                | Escenario de proyección (BSL, LFRT, LMRT, HMIGR, LMIGR, NMIGR) |
| `value`        | `row["projected_old_share_pop"]` | Valor numérico del indicador (float entre 0 y 1)               |


In [20]:
# Preparar registros para projected_old_share_pop
records = []
for _, row in df_proj_old.iterrows():
    records.append({
        "source":       "EUROSTAT_PEP_NUTS_RAW",   # Proyecciones Eurostat (EUROPOP2023)
        "year":         int(row["year"]),          # Año de la proyección
        "region_level": "NAT",                     # Nivel nacional
        "region_code":  "ES",                      # España
        "metric":       "projected_old_share_pop", # Cuota sénior proyectada
        "scenario":     row["scenario"],           # Escenario (BSL, LFRT…)
        "value":        float(row["projected_old_share_pop"])  # Proporción ≥65 / total
    })

# Insertar en la vista
view.insert_many(records)
print("projected_old_share_pop insertado:",
      view.count_documents({"metric":"projected_old_share_pop"}))

projected_old_share_pop insertado: 474


# `projected_dependency_ratio` – Tasa de dependencia proyectada

**Definición**

$$
\text{projected\_dependency\_ratio} = \frac{\text{población proyectada ≥65 años}}{\text{población proyectada en edad de trabajar (15–64)}}
$$

In [21]:
# Población sénior proyectada (65+)
senior_proj = (
    df_proj_norm[df_proj_norm["age"] >= 65]
    .groupby(["year","scenario"], as_index=False)["population"]
    .sum()
    .rename(columns={"population": "old_pop"})
)

# Población en edad de trabajar proyectada (15–64)
work_proj = (
    df_proj_norm[df_proj_norm["age"].between(15, 64)]
    .groupby(["year","scenario"], as_index=False)["population"]
    .sum()
    .rename(columns={"population": "work_pop"})
)

# Unir y calcular la tasa de dependencia proyectada
df_proj_dep = pd.merge(senior_proj, work_proj, on=["year","scenario"])
df_proj_dep["projected_dependency_ratio"] = (
    df_proj_dep["old_pop"] / df_proj_dep["work_pop"]
)

In [22]:
df_proj_dep.head()

,year,scenario,old_pop,work_pop,projected_dependency_ratio
0,2022,BSL,9512343,31278325,0.304119
1,2022,HMIGR,9512343,31278325,0.304119
2,2022,LFRT,9512343,31278325,0.304119
3,2022,LMIGR,9512343,31278325,0.304119
4,2022,LMRT,9512343,31278325,0.304119


| Campo          | Valor                               | Descripción                                                    |
| -------------- | ----------------------------------- | -------------------------------------------------------------- |
| `source`       | `"EUROSTAT_PEP_NUTS_RAW"`           | Colección de proyecciones Eurostat (EUROPOP2023)               |
| `year`         | `row["year"]`                       | Año de la proyección (2022–2100)                               |
| `region_level` | `"NAT"`                             | Nivel territorial: nacional                                    |
| `region_code`  | `"ES"`                              | Código del territorio: España                                  |
| `metric`       | `"projected_dependency_ratio"`      | Indicador: (65+) / (15–64) proyectado                          |
| `scenario`     | `row["scenario"]`                   | Escenario de proyección (BSL, LFRT, LMRT, HMIGR, LMIGR, NMIGR) |
| `value`        | `row["projected_dependency_ratio"]` | Valor numérico de la tasa (float)                              |


In [23]:
# Preparar registros para projected_dependency_ratio
records = []
for _, row in df_proj_dep.iterrows():
    records.append({
        "source":       "EUROSTAT_PEP_NUTS_RAW",      # Proyecciones Eurostat (EUROPOP2023)
        "year":         int(row["year"]),             # Año de la proyección
        "region_level": "NAT",                        # Nivel nacional
        "region_code":  "ES",                         # España
        "metric":       "projected_dependency_ratio", # Tasa de dependencia proyectada
        "scenario":     row["scenario"],              # Escenario (BSL, LFRT, etc.)
        "value":        float(row["projected_dependency_ratio"])  
    })

# Insertar en la vista
view.insert_many(records)
print("projected_dependency_ratio insertado:",
      view.count_documents({"metric":"projected_dependency_ratio"}))

projected_dependency_ratio insertado: 474


# `projected_generational_balance_index_pop` - 

**Definición**  
$$
\text{projected\_generational\_balance\_index\_pop} = \frac{\text{projected\_youth\_share\_pop}}{\text{projected\_old\_share\_pop}}
$$

In [24]:
df_proj_balance = pd.merge(
    df_proj_quota[['year','scenario','projected_youth_share_pop']],
    df_proj_old[['year','scenario','projected_old_share_pop']],
    on=['year','scenario']
)
df_proj_balance['projected_generational_balance_index_pop'] = (
    df_proj_balance['projected_youth_share_pop'] /
    df_proj_balance['projected_old_share_pop']
)

In [25]:
df_proj_balance.head()

,year,scenario,projected_youth_share_pop,projected_old_share_pop,projected_generational_balance_index_pop
0,2022,BSL,0.048939,0.053911,0.90778
1,2022,HMIGR,0.048939,0.053911,0.90778
2,2022,LFRT,0.048939,0.053911,0.90778
3,2022,LMIGR,0.048939,0.053911,0.90778
4,2022,LMRT,0.048939,0.053911,0.90778


| Campo          | Valor                                             | Descripción                                                                       |
| -------------- | ------------------------------------------------- | --------------------------------------------------------------------------------- |
| `source`       | `"EUROSTAT_PEP_NUTS_RAW"`                         | Proyecciones Eurostat (EUROPOP2023)                                               |
| `year`         | `row["year"]`                                     | Año de la proyección (2022–2100)                                                  |
| `region_level` | `"NAT"`                                           | Nivel territorial: nacional                                                       |
| `region_code`  | `"ES"`                                            | Código del territorio: España                                                     |
| `metric`       | `"projected_generational_balance_index_pop"`      | Indicador: ratio cuota juvenil perc. / cuota sénior perc. en población proyectada |
| `scenario`     | `row["scenario"]`                                 | Escenario de proyección (BSL, LFRT, LMRT, HMIGR, LMIGR, NMIGR)                    |
| `value`        | `row["projected_generational_balance_index_pop"]` | Valor numérico del índice (float)                                                 |


In [26]:
# Preparar registros
records = []
for _, row in df_proj_balance.iterrows():
    records.append({
        "source":       "EUROSTAT_PEP_NUTS_RAW",
        "year":         int(row["year"]),
        "region_level": "NAT",
        "region_code":  "ES",
        "metric":       "projected_generational_balance_index_pop",
        "scenario":     row["scenario"],
        "value":        float(row["projected_generational_balance_index_pop"])
    })

# Insertar
view.insert_many(records)
print("projected_generational_balance_index_pop insertado:",
      view.count_documents({"metric":"projected_generational_balance_index_pop"}))

projected_generational_balance_index_pop insertado: 474
